In [7]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State
import base64
import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps


#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from animalShelter import AnimalShelter


test = True


###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "aacuser"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso-Salvare-Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date

app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode())),
    html.H6("Created by Samantha Godwin"),
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Hr(),
    html.Div(className = "row",
             style={"display":"flex"},
                 children=[
                     html.Button(id="submit-button-one", n_clicks = 0, children = "Water_Rescue"),
                     html.Button(id="submit-button-two", n_clicks= 0, children = "Mountain_Rescue"),
                     html.Button(id="submit-button-three", n_clicks=0, children="Disaster_Tracking"),
                     html.Button(id="submit-button-four", n_clicks= 0, children="Reset")
                 ]
        
#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.


    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size = 10,
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('submit-button-one','n_clicks'),Input('submit-button-two','n_clicks'),Input('submit-button-three','n_clicks'),Input('submit-button-four','n_clicks')])
def update_dashboard(button_1,button_2,button_3,button_4):
### FIX ME Add code to filter interactive data table with MongoDB queries

        #TODO - implement filtering options 
        if int(button_1) != 0:
            button_1 = 0
            df = pd.DataFrame.from_records(shelter.read({"breed":{"$in":["Labrador Retriever Mix","Newfoundland","Chesapeake Bay Retriever"]},"sex_upon_outcome":"Intact Female","age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}}))
                                                                                                                        
        if int(button_2) != 0:
            button_2 = 0
            df = pd.DataFrame.from_records(shelter.read({"breed":{"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Husky","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":26,"$lte":156}})) #replace with mountain rescue
        if int(button_3) != 0:
            button_3 = 0
            df = pd.DataFrame.from_records(shelter.read({"breed":{"$in":["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound","Rottweiler"]},"sex_upon_outcome":"Intact Male","age_upon_outcome_in_weeks":{"$gte":20,"$lte":300}})) #replace with disaster rescue
        if int(button_4) != 0:
            button_4 = 0
            df = pd.DataFrame.from_records(shelter.read({})) 
    
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)




@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    return [
        
        dcc.Graph(            
            figure = px.pie(
                data_frame = dff,
                values = "1",
                names = "breed",
                title = "Number of Each Breed"
            )
        )    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style = {"width":"1000px","height":"500px"},center = [30.75,-97.48],zoom = 10,children = [
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position = [30.75,-97.48], children =[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                         ])
            ])
        ])
    ]
#If you completed the Module Six Assignment, you can copy in the code you created here.



app

username and password are aacuser aacuser
connection was successful
